In [1]:
!pip install tensorflow
!pip install pillow
!pip install scipy

In [10]:
import tensorflow as tf
import os
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from image_utils import create_train_test_validation_sets

In [11]:
src_image_path = Path(os.path.join(os.getcwd(), "../data"))
output_path = Path(r"c:\temp")
create_train_test_validation_sets(
    src_image_path,
    output_path)